In [59]:
%matplotlib inline
import scattertext as st
import re, io
from pprint import pprint
import pandas as pd
import numpy as np
import spacy
from scipy.stats import rankdata, hmean, norm
import spacy, nltk
import os, pkgutil, json, urllib
nltk.download('stopwords')
from nltk.corpus import stopwords
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from nltk import word_tokenize
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("&lt;style>.container { width:98% !important; }&lt;/style>"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/giorgiomondauto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
dati_aprile = pd.read_csv('Job_vacancies_aprile_completo.csv', skiprows=1, usecols = [1,2,3],\
                          names = ['Target','Sub_Role','Job_Description'])
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: re.sub('[^a-zA-Z]',' ',x))
dati_aprile.Target = dati_aprile.Target.apply(lambda x: x.lower())
dati_aprile.Job_Description = dati_aprile['Job_Description'].apply(lambda x: word_tokenize(x.lower()))
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if len(i) >3])
Fonti = ['randstad', 'monster', 'infojob', 'technical', 'kelly', 'services', 'italia', 'lavoropi','quanta']
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if i not in Fonti])
list_stopwords = list(set(stopwords.words('italian')))
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if i not in list_stopwords] )
dati_aprile['Job_Description'] = dati_aprile['Job_Description'].apply(lambda x: ' '.join(x))
dati_aprile.groupby('Target').apply(lambda x: x.Job_Description.apply(lambda x: len(''.join(x).split())).sum())

In [ ]:
dati_aprile.

In [66]:
nlp = spacy.load('en')
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(nlp)

In [67]:
corpus = st.CorpusFromParsedDocuments(dati_aprile, category_col='Target', parsed_col='Job_Description').build()

In [77]:
html = produce_scattertext_explorer(corpus,
                                    category='statistici',
                                    category_name='Statistici',
                                    not_category_name='Other',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    transform=st.Scalers.scale,
                                    metadata=dati_aprile['Target'])
file_name = 'Conventions2012ScattertextScale.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 800, height=700)

In [78]:
html = st.produce_scattertext_explorer(corpus,
                                       category='statistici',
                                       category_name='Statistici',
                                       not_category_name='[',
                                       minimum_term_frequency=5,
                                       width_in_pixels=1000,
                                       transform=st.Scalers.log_scale_standardize)
file_name = 'Conventions2012ScattertextLog.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [83]:
html = produce_scattertext_explorer(corpus,
                                    category='statistici',
                                    category_name='Statistici',
                                    not_category_name='Other',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    transform=st.Scalers.percentile,
                                    metadata=dati_aprile['Target'])
file_name = 'Conventions2012ScattertextRankData.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)